# Funciones

In [198]:
import numpy as np

In [199]:
def generar_matriz_qubo(n: int, n_diagonales: int)-> np.array:
    """
    Funcion que genera una matriz de un problema QUBO dado un numero de variables y un numero de diagonales
    Args:
        n (int): numero de variables
        n_diagonales (int): numero de diagonales

    Returns:
        np.array: devuelve la matriz QUBO 
    """

    # Creamos una matriz aleatoria de tamaño nxn
    Q_matrix= np.random.rand(n, n)*2-1  # Rellenamos con números entre -1 y 1
    
    # Hacemos que la matriz sea simétrica en la diagonal inferior únicamente
    for i in range(n):
        for j in range(i + 1, n):  # Solo afecta la parte superior, donde j > i
            Q_matrix[i, j] = 0
    
    for i in range(n):
        for j in range(0, i-(n_diagonales)+1):  # Solo afecta la parte superior, donde j > i
            Q_matrix[i, j] = 0
    
    # Aseguramos que los términos diagonales sean positivos
    for i in range(n):
        Q_matrix[i, i] = abs(Q_matrix[i, i])

    return Q_matrix

In [200]:
n_variables = 20
tau = 500
number_layers = 20
np.random.seed(0)
# Generamos el caso
Q_matrix = generar_matriz_qubo(n_variables, n_variables)
Q_matrix/= np.linalg.norm(Q_matrix)
Q_matrix = np.exp(-tau*Q_matrix)


In [201]:
def tensor_bottom_generator(Q_last_row: np.array)-> np.array:
    n_tensores = 2**(len(Q_last_row)-2)
    i=0
    tensor = np.ones((n_tensores, 1))
    for element in range(len(Q_last_row)-1,1,-1):
        i = i + 1
        aux_tensor = Q_last_row[int(i-1)] * ((np.arange(n_tensores) // 2**(element - 2) % 2).reshape(n_tensores, 1))
        aux_tensor[aux_tensor ==0] = 1
        tensor = tensor * aux_tensor
    tensor = Q_last_row[-1] * np.hstack([tensor, tensor * Q_last_row[-2]])

    n = tensor.shape[0]
    intercalado = np.zeros((n * 2, 2), dtype=tensor.dtype)  # Inicializamos un tensor de ceros de tamaño (2n, 2)
    # Colocar las filas de [1, 1] en las posiciones impares
    intercalado[::2] = [1, 1]
    # Colocar las filas originales en las posiciones pares
    intercalado[1::2] = tensor
    tensor = intercalado.reshape([2]*(len(Q_last_row)))
    return tensor

In [202]:
def tensor_bottom_generator(Q_last_row: np.array) -> np.array:
    # Número de tensores: 2^(len(Q_last_row) - 2)
    n_tensores = 2 ** (len(Q_last_row) - 2)
    
    # Inicializamos el tensor con 1s
    tensor = np.ones((n_tensores, 1), dtype=Q_last_row.dtype)
    
    # Calculamos el tensor con las multiplicaciones necesarias
    for element in range(len(Q_last_row) - 1, 1, -1):
        # Operación eficiente para generar aux_tensor
        aux_tensor = Q_last_row[element - 1] * ((np.arange(n_tensores) // 2 ** (element - 2) % 2).reshape(n_tensores, 1))
        aux_tensor[aux_tensor == 0] = 1  # Asignamos 1 donde los elementos son 0
        tensor *= aux_tensor  # Multiplicamos directamente el tensor
    
    # Actualizamos tensor con el valor final de Q_last_row[-1] y Q_last_row[-2]
    tensor = Q_last_row[-1] * np.hstack([tensor, tensor * Q_last_row[-2]])

    # Intercalado: creando un tensor final con [1, 1] intercalado entre las filas
    n = tensor.shape[0]
    intercalado = np.zeros((n * 2, 2), dtype=tensor.dtype)

    # Colocamos las filas de [1, 1] en las posiciones impares
    intercalado[::2] = [1, 1]
    
    # Colocamos las filas originales del tensor en las posiciones pares
    intercalado[1::2] = tensor
    
    # Redimensionamos el tensor final a la forma deseada, (2, 2, 2, ..., 2)
    final_shape = [2] * len(Q_last_row)
    tensor = intercalado.reshape(final_shape)
    
    return tensor

In [209]:
in1 = time()
tensor = tensor_bottom_generator(Q_matrix[-1])
print(time()-in1)
#print(tensor)

0.047708988189697266


In [204]:
import numpy as np

# Vector original
vector = np.zeros((8,1))

# Crear un vector de unos con las mismas dimensiones
unos = np.ones_like(vector)

# Apilar los dos vectores y reordenar las filas
resultado = np.vstack((vector, np.ones_like(vector))).reshape(-1, 1)[np.argsort(np.tile(np.arange(len(vector)), 2))]

print(resultado)

[[0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]]
